In [1]:
import tensorflow as tf
import glob
import xmltodict
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
import numpy as np
from PIL import ImageDraw,Image
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils

Using TensorFlow backend.


In [52]:
##Validation is not working
##This is an amazing code by @aryapande

image_dim = 228
input_dim = image_dim
input_shape = (image_dim, image_dim, 3)
dropout_rate = 0.5
alpha = 0.2
num_classes = 1  ##This was set to 3 first. but was not working
pred_vector_length = 4 + num_classes

In [53]:
images = []
for x in glob.glob('datasets/training_images/*.jpg'):
    image = (Image.open(x)).resize((image_dim,image_dim))
    image = np.asarray(image)
    images.append(image)

In [54]:
bboxes = []
classes = []
classes_raw=[]
xml_data = glob.glob( 'datasets/training_images/*.xml' )
for file in xml_data:
    x = xmltodict.parse(open(file,'rb'))
    bndbox = x[ 'annotation' ][ 'object' ][ 'bndbox' ]
    bounding_box = [None] * 4
    bounding_box[0] = int(bndbox['xmin']) / image_dim
    bounding_box[1] = int(bndbox['ymin']) / image_dim
    bounding_box[2] = int(bndbox['xmax']) / image_dim
    bounding_box[3] = int(bndbox['ymax']) / image_dim
    bboxes.append(bounding_box)
    classes_raw.append(x['annotation']['object']['name'])

In [55]:
boxes = np.array(bboxes)
encoder = LabelBinarizer() #like 1 hot encoder but it works on string classes too
classes_onehot = encoder.fit_transform(classes_raw)

Y = np.concatenate([boxes, classes_onehot], axis=1)
X = np.array(images)

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.1)

In [56]:
def custom_loss( y_true , y_pred ):
    mse = tf.losses.mean_squared_error( y_true , y_pred )
    return mse

In [57]:
model_layers = [
	keras.layers.Conv2D(16, kernel_size=(3, 3), strides=1, input_shape=input_shape),
    keras.layers.LeakyReLU( alpha=alpha ) ,
    keras.layers.Conv2D(16, kernel_size=(3, 3), strides=1 ),
    keras.layers.LeakyReLU( alpha=alpha ) ,
    keras.layers.MaxPooling2D( pool_size=( 2 , 2 ) ),

    keras.layers.Conv2D(32, kernel_size=(3, 3), strides=1),
    keras.layers.LeakyReLU( alpha=alpha ) ,
    keras.layers.Conv2D(32, kernel_size=(3, 3), strides=1),
    keras.layers.LeakyReLU( alpha=alpha ) ,
    keras.layers.MaxPooling2D( pool_size=( 2 , 2 ) ),

    keras.layers.Conv2D(64, kernel_size=(3, 3), strides=1),
    keras.layers.LeakyReLU( alpha=alpha ) ,
    keras.layers.Conv2D(64, kernel_size=(3, 3), strides=1),
    keras.layers.LeakyReLU( alpha=alpha ) ,
    keras.layers.MaxPooling2D( pool_size=( 2 , 2 ) ),

    keras.layers.Conv2D(128, kernel_size=(3, 3), strides=1),
    keras.layers.LeakyReLU( alpha=alpha ) ,
    keras.layers.Conv2D(128, kernel_size=(3, 3), strides=1),
    keras.layers.LeakyReLU( alpha=alpha ) ,
    keras.layers.MaxPooling2D( pool_size=( 2 , 2 ) ),

    keras.layers.Conv2D(256, kernel_size=(3, 3), strides=1),
    keras.layers.LeakyReLU( alpha=alpha ) ,
    keras.layers.Conv2D(256, kernel_size=(3, 3), strides=1),
    keras.layers.LeakyReLU( alpha=alpha ) ,
    keras.layers.MaxPooling2D( pool_size=( 2 , 2 ) ),

    keras.layers.Flatten() ,

    keras.layers.Dense( 1240 ) ,
    keras.layers.LeakyReLU( alpha=alpha ) ,
    keras.layers.Dense( 640 ) ,
    keras.layers.LeakyReLU( alpha=alpha ) ,
    keras.layers.Dense( 480 ) ,
    keras.layers.LeakyReLU( alpha=alpha ) ,
    keras.layers.Dense( 120 ) ,
    keras.layers.LeakyReLU( alpha=alpha ) ,
    keras.layers.Dense( 62 ) ,
    keras.layers.LeakyReLU( alpha=alpha ) ,

    keras.layers.Dense( pred_vector_length ),
    keras.layers.LeakyReLU( alpha=alpha ) ,
]

In [58]:
model = keras.Sequential(model_layers)
model.compile(optimizer=keras.optimizers.Adam( lr=0.0001 ),loss=custom_loss,metrics=['accuracy'])

In [59]:
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_91 (Conv2D)           (None, 226, 226, 16)      448       
_________________________________________________________________
leaky_re_lu_143 (LeakyReLU)  (None, 226, 226, 16)      0         
_________________________________________________________________
conv2d_92 (Conv2D)           (None, 224, 224, 16)      2320      
_________________________________________________________________
leaky_re_lu_144 (LeakyReLU)  (None, 224, 224, 16)      0         
_________________________________________________________________
max_pooling2d_46 (MaxPooling (None, 112, 112, 16)      0         
_________________________________________________________________
conv2d_93 (Conv2D)           (None, 110, 110, 32)      4640      
_________________________________________________________________
leaky_re_lu_145 (LeakyReLU)  (None, 110, 110, 32)    

In [60]:
model.fit(x_train,y_train,validation_data=( x_test , y_test ),epochs=100,batch_size=3)

Train on 18 samples, validate on 2 samples
Epoch 1/100
18/18 [==============================] - 2s 88ms/step - loss: 0.4605 - accuracy: 0.4444 - val_loss: 0.3383 - val_accuracy: 0.5000
Epoch 2/100
18/18 [==============================] - 1s 62ms/step - loss: 0.1706 - accuracy: 0.5000 - val_loss: 0.1076 - val_accuracy: 0.5000
Epoch 3/100
18/18 [==============================] - 1s 64ms/step - loss: 0.0813 - accuracy: 0.4444 - val_loss: 0.0982 - val_accuracy: 0.0000e+00
Epoch 4/100
18/18 [==============================] - 1s 62ms/step - loss: 0.0452 - accuracy: 0.7222 - val_loss: 0.0364 - val_accuracy: 0.5000
Epoch 5/100
18/18 [==============================] - 1s 60ms/step - loss: 0.0322 - accuracy: 0.5000 - val_loss: 0.0543 - val_accuracy: 0.0000e+00
Epoch 6/100
18/18 [==============================] - 1s 62ms/step - loss: 0.0131 - accuracy: 0.7778 - val_loss: 0.0271 - val_accuracy: 0.0000e+00
Epoch 7/100
18/18 [==============================] - 1s 60ms/step - loss: 0.0110 - accuracy: 

18/18 [==============================] - 1s 64ms/step - loss: 3.2431e-04 - accuracy: 0.8889 - val_loss: 0.0226 - val_accuracy: 0.0000e+00
Epoch 57/100
18/18 [==============================] - 1s 62ms/step - loss: 5.0611e-04 - accuracy: 0.8889 - val_loss: 0.0270 - val_accuracy: 0.0000e+00
Epoch 58/100
18/18 [==============================] - 1s 61ms/step - loss: 5.0621e-04 - accuracy: 1.0000 - val_loss: 0.0283 - val_accuracy: 0.0000e+00
Epoch 59/100
18/18 [==============================] - 1s 61ms/step - loss: 5.5590e-04 - accuracy: 0.8889 - val_loss: 0.0291 - val_accuracy: 0.0000e+00
Epoch 60/100
18/18 [==============================] - 1s 62ms/step - loss: 2.8439e-04 - accuracy: 0.8333 - val_loss: 0.0268 - val_accuracy: 0.0000e+00
Epoch 61/100
18/18 [==============================] - 1s 61ms/step - loss: 4.3049e-04 - accuracy: 0.8889 - val_loss: 0.0253 - val_accuracy: 0.0000e+00
Epoch 62/100
18/18 [==============================] - 1s 62ms/step - loss: 2.8344e-04 - accuracy: 1.0000 - 

In [61]:
model.save( 'model.h5')

boxes = model.predict(x_test)

In [62]:
for i in range(boxes.shape[0]):
    b = boxes[i, 0: 4] * input_dim
    img = x_test[i] * 255
    source_img = Image.fromarray(img.astype(np.uint8), 'RGB')
    draw = ImageDraw.Draw(source_img)
    draw.rectangle(b, outline="black")
    source_img.save('datasets/result_images/image_{}.png'.format(i + 1), 'png')

In [63]:
def class_accuracy( target_classes , pred_classes ):
    target_classes = np.argmax( target_classes , axis=1 )
    pred_classes = np.argmax( pred_classes , axis=1 )
    return ( target_classes == pred_classes ).mean()

In [64]:
target_boxes = y_test * input_dim
pred = model.predict( x_test )
pred_boxes = pred[ ... , 0 : 4 ] * input_dim
pred_classes = pred[ ... , 4 : ]
print( 'Class Accuracy is {} %'.format( class_accuracy( y_test[ ... , 4 : ] , pred_classes ) * 100 ))

Class Accuracy is 100.0 %
